### Importaciones

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import janitor 

/home/gresuto/mambaforge/envs/curso-np-pd/lib/python3.11/site-packages/janitor/utils.py:84: FutureWarning: PandasArray has been renamed NumpyExtensionArray. Use that instead. This alias will be removed in a future version.
  @_expand_grid.register(pd.arrays.PandasArray)


### Carga de datos y tratamientos de valores faltantes o inconscistencias

#### Carga de datos

In [2]:
data_raw = pd.read_csv('euro2024_players.csv')
data_raw

,Name,Position,Age,Club,Height,Foot,Caps,Goals,MarketValue,Country
0,Marc-André ter Stegen,Goalkeeper,32,FC Barcelona,187,right,40,0,28000000,Germany
1,Manuel Neuer,Goalkeeper,38,Bayern Munich,193,right,119,0,4000000,Germany
2,Oliver Baumann,Goalkeeper,34,TSG 1899 Hoffenheim,187,right,0,0,3000000,Germany
3,Nico Schlotterbeck,Centre-Back,24,Borussia Dortmund,191,left,12,0,40000000,Germany
4,Jonathan Tah,Centre-Back,28,Bayer 04 Leverkusen,195,right,25,0,30000000,Germany
...,...,...,...,...,...,...,...,...,...,...
618,Adam Hlozek,Second Striker,21,Bayer 04 Leverkusen,188,right,31,2,12000000,Czech Republic
619,Patrik Schick,Centre-Forward,28,Bayer 04 Leverkusen,191,left,37,18,22000000,Czech Republic
620,Mojmír Chytil,Centre-Forward,25,SK Slavia Prague,187,-,12,4,6500000,Czech Republic
621,Jan Kuchta,Centre-Forward,27,AC Sparta Prague,185,right,20,3,5000000,Czech Republic


#### Limpiando datos

In [3]:

data_raw['Foot'] = data_raw['Foot'].fillna('right')
data_raw['Foot'] = data_raw['Foot'].str.replace("-", "right", case=False, regex=False)

data = (
    data_raw.rename_columns(
    {
        'Name' : 'Nombre',
        'Position' : 'Posicion',
        'Age' : 'Edad',
        'Club' : 'Club',
        'Height' : 'Estatura',
        'Foot' : 'Pierna_Dom',
        'Caps' : 'Partidos',
        'Goals' : 'Goles',
        'MarketValue' : 'Valor_de_Mercado',
        'Country' : 'Pais'    
    }
    
)

    
)
data

,Nombre,Posicion,Edad,Club,Estatura,Pierna_Dom,Partidos,Goles,Valor_de_Mercado,Pais
0,Marc-André ter Stegen,Goalkeeper,32,FC Barcelona,187,right,40,0,28000000,Germany
1,Manuel Neuer,Goalkeeper,38,Bayern Munich,193,right,119,0,4000000,Germany
2,Oliver Baumann,Goalkeeper,34,TSG 1899 Hoffenheim,187,right,0,0,3000000,Germany
3,Nico Schlotterbeck,Centre-Back,24,Borussia Dortmund,191,left,12,0,40000000,Germany
4,Jonathan Tah,Centre-Back,28,Bayer 04 Leverkusen,195,right,25,0,30000000,Germany
...,...,...,...,...,...,...,...,...,...,...
618,Adam Hlozek,Second Striker,21,Bayer 04 Leverkusen,188,right,31,2,12000000,Czech Republic
619,Patrik Schick,Centre-Forward,28,Bayer 04 Leverkusen,191,left,37,18,22000000,Czech Republic
620,Mojmír Chytil,Centre-Forward,25,SK Slavia Prague,187,right,12,4,6500000,Czech Republic
621,Jan Kuchta,Centre-Forward,27,AC Sparta Prague,185,right,20,3,5000000,Czech Republic


#### Traduciendo posiciones al espanol

In [4]:
data['Posicion'] = data['Posicion'].replace(
    {
        
        'Centre-Back' : 'Defensa-Central',
        'Central Midfield' :  'Centrocampista-Central',
        'Centre-Forward': 'Delantero-Centro',
        'Goalkeeper' : 'Portero',
        'Defensive Midfield' : 'Centrocampista-Defensivo',
        'Attacking Midfield' : 'Centrocampista-Ofensivo',
        'Left Winger' : 'Extremo-Izquierdo',
        'Right-Back' : 'Lateral-Derecho',
        'Right Winger' :  'Extremo-Derecho',
        'Left-Back' : 'Lateral-Izquierdo',
        'Right Midfield' : 'Interior-Derecho',
        'Second Striker': 'Segundo-Delantero',
        'Left Midfield': 'Interior-Izquierdo'
    }
)


In [5]:
(
    data.isnull()
    .pipe(
        lambda df: df[df==True].count()
    )
)

Nombre              0
Posicion            0
Edad                0
Club                0
Estatura            0
Pierna_Dom          0
Partidos            0
Goles               0
Valor_de_Mercado    0
Pais                0
dtype: int64

### Preguntas a priori

#### ¿Como está distribuido la participación por posición?

In [6]:
data['Posicion'].value_counts()

Posicion
Defensa-Central             125
Centrocampista-Central       81
Delantero-Centro             77
Portero                      72
Centrocampista-Defensivo     50
Centrocampista-Ofensivo      48
Extremo-Izquierdo            44
Lateral-Derecho              43
Extremo-Derecho              35
Lateral-Izquierdo            33
Interior-Derecho              6
Segundo-Delantero             5
Interior-Izquierdo            4
Name: count, dtype: int64

#### ¿Cual es la distribucion de participacion de clubes?

In [7]:
data.Club.value_counts().head(25)

Club
Manchester City        13
Inter Milan            13
FC Barcelona           12
Real Madrid            12
Paris Saint-Germain    12
RB Leipzig             11
Bayern Munich          11
Bayer 04 Leverkusen    10
Arsenal FC             10
Liverpool FC            9
Juventus FC             9
SK Slavia Prague        9
AS Roma                 9
VfL Wolfsburg           8
AC Milan                8
Fenerbahce              8
Manchester United       8
Bologna FC 1909         8
Atlético de Madrid      7
Feyenoord Rotterdam     7
Brentford FC            7
Shakhtar Donetsk        7
Torino FC               6
RSC Anderlecht          6
Real Sociedad           6
Name: count, dtype: int64

#### Mayores goleadores internacionales de la competicion

In [8]:
numero_ranking = [1]
data.select_columns(['Nombre', 'Goles']).sort_values('Goles', ascending=False).set_index(data.index+1).head(30)

,Nombre,Goles
1,Cristiano Ronaldo,128
2,Romelu Lukaku,83
3,Robert Lewandowski,82
4,Harry Kane,63
5,Olivier Giroud,57
6,Aleksandar Mitrovic,57
7,Kylian Mbappé,47
8,Andriy Yarmolenko,46
9,Thomas Müller,45
10,Memphis Depay,45


#### Jugadores mas valiosos

In [9]:
data.select_columns(["Nombre", 'Valor_de_Mercado' ]).sort_values('Valor_de_Mercado', ascending=False).set_index(data.index+1).head(20)

,Nombre,Valor_de_Mercado
1,Jude Bellingham,180000000
2,Kylian Mbappé,180000000
3,Phil Foden,150000000
4,Bukayo Saka,140000000
5,Florian Wirtz,130000000
6,Rodri,120000000
7,Declan Rice,120000000
8,Jamal Musiala,120000000
9,Eduardo Camavinga,100000000
10,Harry Kane,100000000


### Preguntas de primer nivel

#### Construyendo Dataframe por equipos

In [10]:
data_equipos = (
    
    data.groupby(['Pais'])
    .sum()
    .drop(columns=['Nombre', 'Pierna_Dom', 'Club'])
    .assign(Total_Jugadores= data['Pais'].value_counts()) 
                )

data_equipos['Edad'] = data_equipos['Edad']/data_equipos['Total_Jugadores']

data_equipos['Estatura'] = data_equipos['Estatura']/data_equipos['Total_Jugadores']


data_equipos

,Posicion,Edad,Estatura,Partidos,Goles,Valor_de_Mercado,Total_Jugadores
Pais,,,,,,,
Albania,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.307692,183.615385,679,40,111600000,26
Austria,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.807692,183.192308,613,93,235500000,26
Belgium,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.680000,949,177,584500000,25
Croatia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,184.115385,1152,147,327700000,26
Czech Republic,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,25.307692,185.538462,405,65,193900000,26
Denmark,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,186.269231,1071,135,415500000,26
England,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.076923,182.461538,651,100,1515000000,26
France,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.440000,836,192,1234000000,25
Georgia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.153846,184.500000,750,64,160150000,26


#### Rankings

##### Ranking de estatura promedio de los equipos

In [11]:
altura_rank = equipos_clean.loc[:,['Country', 'Height']].sort_values('Height',ascending=False).set_index(data.index+1)
altura_rank

NameError: name 'equipos_clean' is not defined

### Preguntas de 2do Nivel

#### Agregando variables necesarias

##### Agregando Promedio de goles e Indice de Distribucion de goles

In [ ]:
data_equipos = (
    data_equipos
    .assign(Prom_Goles= data_equipos['Goles']/data_equipos['Partidos'])
    .assign(Ind_Dist_Goles= data
            .select_columns(['Pais','Goles'])
            .groupby('Pais')
            .std()
    )
)

data_equipos

,Posicion,Edad,Estatura,Partidos,Goles,Valor_de_Mercado,Total_Jugadores,Prom_Goles,Ind_Dist_Goles
Pais,,,,,,,,,
Albania,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.307692,183.615385,679,40,111600000,26,0.058910,1.881080
Austria,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.807692,183.192308,613,93,235500000,26,0.151713,8.178866
Belgium,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.680000,949,177,584500000,25,0.186512,16.989997
Croatia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,184.115385,1152,147,327700000,26,0.127604,9.103592
Czech Republic,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,25.307692,185.538462,405,65,193900000,26,0.160494,4.235564
Denmark,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.692308,186.269231,1071,135,415500000,26,0.126050,8.366692
England,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.076923,182.461538,651,100,1515000000,26,0.153610,12.295340
France,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,26.880000,184.440000,836,192,1234000000,25,0.229665,15.949713
Georgia,PorteroPorteroPorteroDefensa-CentralDefensa-Ce...,27.153846,184.500000,750,64,160150000,26,0.085333,3.797165


##### Agregando posiciones de los jugadores

In [70]:
posiciones_df = data.pivot_table(index= 'Pais', columns='Posicion', values='Nombre', aggfunc='count').reset_index()
posiciones_df = posiciones_df.fillna(0)


posiciones_resumen_df = (
    pd.DataFrame({
    'Pais' : posiciones_df['Pais']
    
    })
    .assign(
        Defensas= posiciones_df['Defensa-Central']
        + posiciones_df['Lateral-Derecho']
        + posiciones_df['Lateral-Izquierdo']
        
    ) 
    .assign(
        Centrocampista= posiciones_df['Centrocampista-Central'] 
        + posiciones_df['Centrocampista-Defensivo'] 
        + posiciones_df['Centrocampista-Ofensivo']
        + posiciones_df['Interior-Derecho']
        + posiciones_df['Interior-Izquierdo']
    )
    .assign(
        Atacantes= posiciones_df['Delantero-Centro'] 
        + posiciones_df['Extremo-Derecho'] 
        + posiciones_df['Extremo-Izquierdo']
        + posiciones_df['Segundo-Delantero']
    )
)



posiciones_df
posiciones_resumen_df.sort_values(by='Atacantes',ascending=False)
#posiciones_df['Centrocampista-Central'] + posiciones_df['Centrocampista-Defensivo'] + posiciones_df['Centrocampista-Ofensivo']
#data.select_columns(['Pais', 'Posicion']).groupby(['Pais']).value_counts()

,Pais,Defensas,Centrocampista,Atacantes
5,Denmark,9.0,6.0,8.0
7,France,8.0,6.0,8.0
8,Georgia,7.0,8.0,8.0
22,Turkiye,8.0,7.0,8.0
21,Switzerland,7.0,8.0,8.0
15,Romania,8.0,7.0,8.0
23,Ukraine,10.0,6.0,7.0
17,Serbia,8.0,8.0,7.0
2,Belgium,8.0,7.0,7.0
3,Croatia,8.0,8.0,7.0


#### Rankings

##### Ranking promedio de goles